In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import json
import datetime
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import (Mail, Attachment, FileContent, FileName, FileType, Disposition, ContentId, To)
import base64
from simple_salesforce import Salesforce,SalesforceMalformedRequest

In [ ]:
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Get sendgrid_api_token from Snowflake secrets
use schema PYTHON_WORKLOADS.email_reports;
CREATE OR REPLACE function sendgrid_api_key_secret()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.9
HANDLER = 'get_secret_1'
EXTERNAL_ACCESS_INTEGRATIONS = (SENDGRID_APIS_ACCESS_INTEGRATION_EMAIL)
SECRETS = ('cred' = sendgrid_dev_API_TOKEN )
AS
$$
import _snowflake
def get_secret_1():
   secret_type = _snowflake.get_generic_secret_string('cred')
   return secret_type
$$;
grant usage on function sendgrid_api_key_secret() to role svc_engineer_prd;

In [ ]:
# Store sf api token in variable
api_secret_query = "select PYTHON_WORKLOADS.email_reports.sendgrid_api_key_secret()"
sendgrid_api_key = session.sql(api_secret_query).collect()[0][0]

In [ ]:
current_env="UAT"
errorlist = []
failed_keys = []
failed_calllogoid = []
error_msg = []
error_field = []
error_concat_list = []

In [ ]:
# Connect to Salesforce - UAT
if current_env=="PRD":
    sf = Salesforce (username=sf_uname, password=sf_password, security_token=sf_token) 
else:
    sf = Salesforce (username="", password="", security_token="",domain='heartlandpaymentsystems--qauat.sandbox.my') 

In [ ]:
def records_insert(sf,records):
    try:
        results = sf.bulk.Case.insert(records)
    except SalesforceMalformedRequest as e:
        logging.info(f"Record insertion failed due to malformed request: {e}")
        return [],[]

    successful_records=[]
    failed_records=[]

    for record,result in zip(records,results):
        if result['success']:
            successful_records.append(record)
        else:
            failed_records.append((record,result['errors']))
            # logging.info(failed_records)

    return successful_records,failed_records

In [ ]:
-- Query to fetch new call log entries
select call.merchantsequencekey,call.callername,call.calllogoid,note.note,
merch.fedtaxid,call.entereddate,calltype.description as calltypedesc,detailtype.description as detailtypedesc,
SHA2(CONCAT(call.merchantsequencekey,call.calllogoid,call.entereddate)) as hash_value, 'Closed' as status
from hcsdb_stg.dbo.HPSTCALLLOG call
inner join hcsdb_stg.dbo.hpstcalllognotes note
on call.calllogoid=note.calllogoid
inner join hcsdb_stg.dbo.merchants merch
on call.merchantsequencekey=merch.sequencekey
inner join hcsdb_stg.DBO.HPSTCLCALLTYPE calltype
on call.CallTypeOID=calltype.CallTypeOID
inner join hcsdb_stg.DBO.HPSTCLDETAIL detailtype
on call.DetailTypeOID=detailtype.DetailTypeOID
where  merch.fedtaxid is not null and hash_value not in (select distinct hash_value from PYTHON_WORKLOADS.SERVICING.hcsdb_caselog) and isfromsalesforce=False and call.entereddate between DATEADD(HOUR, -48, current_timestamp()) and current_timestamp();

In [ ]:
source_df = cell5.to_pandas()
if len(source_df.index)>0:
    case_df=source_df
    case_df['CALLLOGOID']='Servicing Site Call Log '+case_df['CALLLOGOID'].astype('str')
    case_df['Account'] = '{"Fed_Tax_ID__c":"'+ case_df['FEDTAXID'].astype('str')+'"}'
    case_df['Account'] = case_df['Account'].apply(json.loads)
    case_df['Gateway_Account__r'] = '{"Sequencekey__c":"'+ case_df['MERCHANTSEQUENCEKEY'].astype('str')+'"}'
    case_df['Gateway_Account__r'] = case_df['Gateway_Account__r'].apply(json.loads)
    case_df = case_df.rename(columns={'CALLLOGOID': 'Subject', 'NOTE': 'Description', 'CALLERNAME':'Caller__c','CALLTYPEDESC':'Call_Reason_for_Integration__c','DETAILTYPEDESC':'Detail_Description_for_Integration__c','STATUS':'Status'})
    case_final_df = case_df[['Account','Gateway_Account__r','Subject','Description','Caller__c','Call_Reason_for_Integration__c','Detail_Description_for_Integration__c','Status']]
    
    case_json = case_final_df.to_json(orient='records')
    insert_data_list = json.loads(case_json)
    
    if len(insert_data_list)>0:
        successful_records,failed_records=records_insert(sf,insert_data_list)
        if successful_records:
            initial_record_count=len(successful_records)
        else:
            initial_record_count=0

    for record,errors in failed_records:
        for error_val in errors:
            error_msg.append(error_val['message'])
            error_field.append(error_val['fields'])
            failed_keys.append(record['Gateway_Account__r']['Sequencekey__c'])
            failed_calllogoid.append(record['Subject'])

    error_concat_list=[str(field)+' - '+str(value) + ' - ' + str(key) + ' - ' + str(logid) for field,value,key,logid in zip(error_field,error_msg,failed_keys,failed_calllogoid)]
    concat_key_list = [str(key)+str(calllog) for key,calllog in zip(failed_keys,failed_calllogoid)]
    concat_key_list = list(set(concat_key_list))
    concat_key_list_df = pd.DataFrame({'concat_key_list':concat_key_list})
    error_concat_list = list(set(error_concat_list))
    
    case_hash_df=source_df[['CALLLOGOID','HASH_VALUE','MERCHANTSEQUENCEKEY']]
    case_hash_df.loc[:,'concat_key_list']=case_hash_df['MERCHANTSEQUENCEKEY'].astype(str)+''+case_hash_df['CALLLOGOID'].astype(str)
    df_merge=pd.merge(case_hash_df,concat_key_list_df,on='concat_key_list',how='left',indicator=True)
    case_hash_df_filtered=df_merge[df_merge['_merge']=='left_only'].drop(columns='_merge')
    case_hash_df_filtered=case_hash_df_filtered[['HASH_VALUE']]
    print(f"No. of successful entries: {initial_record_count}")
    print(f"No. of failed entries: {len(failed_keys)}")

else:
    pass

In [ ]:
if len(source_df.index)>0:
    case_hash_df=source_df[['CALLLOGOID','HASH_VALUE','MERCHANTSEQUENCEKEY']]
    case_hash_df.loc[:,'concat_key_list']=case_hash_df['MERCHANTSEQUENCEKEY'].astype(str)+''+case_hash_df['CALLLOGOID'].astype(str)
    df_merge=pd.merge(case_hash_df,concat_key_list_df,on='concat_key_list',how='left',indicator=True)
    case_hash_df_filtered=df_merge[df_merge['_merge']=='left_only'].drop(columns='_merge')
    case_hash_df_filtered=case_hash_df_filtered[['HASH_VALUE']]
else:
    pass

In [ ]:
if len(case_hash_df_filtered.index)>0:
    print(case_hash_df_filtered)
else:
    pass

In [ ]:
if len(case_hash_df_filtered.index)>0:
    session.write_pandas(case_hash_df_filtered,"HCSDB_CASELOG",database="PYTHON_WORKLOADS",schema="SERVICING",auto_create_table=False,overwrite=False)
else:
    pass

In [ ]:
if(len(error_concat_list)>0):
    cc_email = ''
    
    sub1  = "HCSDB Snowflake to SalesForce :: Case Objects :: Failed Sequence keys"
    body1 = """Case objects were not created on the previous run -  """ + pd.Timestamp("today").strftime("%m-%d-%Y") + f""". Please refer to the attached file for more information."""

    message = Mail(
        from_email='bi@heartland.us',
        to_emails=[To('arvindraj.rajakumar@e-hps.com')],
        subject=sub1,
        plain_text_content=body1
    )
    if cc_email:
        message.add_cc(cc_email)

    
    failed_id_df = pd.DataFrame({'Field - Value - Sequence Key - Call log ID':error_concat_list})
    base64_csv = base64.b64encode(failed_id_df.to_csv(index=False).encode()).decode()

    message.attachment = Attachment(FileContent(base64_csv),
                        FileName('Failed_Case_entires.csv'),
                        FileType('text/csv'),
                        Disposition('attachment'),
                        ContentId('dataframe'))


    sg = SendGridAPIClient(sendgrid_api_key)
    response = sg.send(message)